In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime
import qiskit

# --- Classical Chemistry Imports (PySCF) ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer

# --- Qiskit Algorithm Imports ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B

# --- Qiskit Aer Simulator Import ---
# We use the high-performance local Estimator from Qiskit Aer
from qiskit_aer.primitives import Estimator as AerEstimator

# --- Configuration Parameters ---
BASIS_SET = "sto-3g"
BOND_LENGTH = 0.9578
BOND_ANGLE = 104.478

def get_h2o_geometry(bond_length=BOND_LENGTH, angle_deg=BOND_ANGLE):
    """Returns the H2O geometry for the given parameters."""
    angle_rad = angle_deg * np.pi / 180
    x = bond_length * np.sin(angle_rad / 2)
    y = bond_length * np.cos(angle_rad / 2)
    return [("O", (0.0, 0.0, 0.0)), ("H", (x, y, 0.0)), ("H", (-x, y, 0.0))]

# --- Main Execution ---
def main():
    print(f"💧 H₂O Ground State Analysis (Qiskit v{qiskit.__version__})")
    print("   (Local Simulator vs. Classical Methods)")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    results = {}
    geometry = get_h2o_geometry()

    # --- 1. Classical Calculations (PySCF) ---
    print("--- Running Classical Calculations (PySCF) ---")
    classical_start_time = time.time()
    mol = gto.M(atom=geometry, basis=BASIS_SET, unit='angstrom', verbose=0)
    
    # Unrestricted Hartree-Fock (UHF)
    mf_uhf = scf.UHF(mol).run()
    results['UHF'] = mf_uhf.e_tot
    print(f"  UHF Energy:              {results['UHF']:.8f} Hartree")
    
    # Full Configuration Interaction (FCI) - The exact answer for the basis set
    mf_rhf = scf.RHF(mol).run()
    results['FCI'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    print(f"  FCI (Exact) Energy:      {results['FCI']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - classical_start_time:.2f}s\n")

    # --- 2. Quantum VQE Calculation (Local Simulator) ---
    print("--- Running VQE Calculation (Qiskit Aer Simulator) ---")
    vqe_start_time = time.time()
    
    # Setup the chemistry problem for Qiskit Nature
    atom_str = "; ".join([f"{atom[0]} {coords[0]} {coords[1]} {coords[2]}" for atom, coords in geometry])
    driver = PySCFDriver(atom=atom_str, unit=DistanceUnit.ANGSTROM, charge=0, spin=0, basis=BASIS_SET)
    problem = driver.run()

    # Define a small active space to keep the calculation fast
    transformer = ActiveSpaceTransformer(num_electrons=2, num_spatial_orbitals=2)
    problem = FreezeCoreTransformer().transform(problem)
    problem = transformer.transform(problem)

    mapper = ParityMapper(num_particles=problem.num_particles)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    # Setup VQE components
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper)
    optimizer = L_BFGS_B()
    
    # A callback to store intermediate energy values from the optimizer
    intermediate_values = []
    def callback(eval_count, parameters, mean, std):
        intermediate_values.append(mean)

    # Use the local AerEstimator for a fast, noiseless simulation
    estimator = AerEstimator()
    
    vqe = VQE(estimator, ansatz, optimizer, callback=callback)
    vqe_calc_result = vqe.compute_minimum_eigenvalue(qubit_op)
    
    # Interpret the result to get the final energy
    vqe_energy = problem.interpret(vqe_calc_result).total_energies[0].real
    results['VQE'] = vqe_energy
    
    print(f"  VQE Energy (Simulated):  {results['VQE']:.8f} Hartree")
    print(f"VQE simulation finished in {time.time() - vqe_start_time:.2f}s")

    # --- 3. Final Summary and Plot ---
    print("\n" + "="*55)
    print("               Final Energy Summary")
    print("="*55)
    print(f"  UHF Energy:              {results['UHF']:.8f} Hartree")
    print(f"  VQE Energy (Simulated):  {results['VQE']:.8f} Hartree")
    print(f"  FCI (Full Exact) Energy: {results['FCI']:.8f} Hartree")
    print("-" * 55)
    vqe_error = abs(results['VQE'] - results['FCI'])
    print(f"  Error (VQE vs. Full FCI):  {vqe_error:.8f} Hartree")
    print("="*55)
    
    # Plot the convergence of the optimizer
    plt.figure(figsize=(8, 5))
    plt.plot(intermediate_values, color='dodgerblue', lw=2, label='VQE Energy')
    plt.axhline(y=results['FCI'], color='black', linestyle='--', label='FCI (Exact) Energy')
    plt.ylabel('Energy (Hartree)')
    plt.xlabel('Optimizer Iteration')
    plt.title('VQE Convergence for H₂O (Local Simulator)')
    plt.legend()
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    main()

💧 H₂O Ground State Analysis (Qiskit v2.1.2)
   (Local Simulator vs. Classical Methods)
Started at: 2025-09-17 12:30:29

--- Running Classical Calculations (PySCF) ---
  UHF Energy:              -74.96302717 Hartree
  FCI (Exact) Energy:      -75.01258567 Hartree
Classical calculations finished in 0.64s

--- Running VQE Calculation (Qiskit Aer Simulator) ---


/tmp/ipykernel_487760/301308756.py:125: DeprecationWarning: Estimator has been deprecated as of Aer 0.15, please use EstimatorV2 instead.
  main()
/tmp/ipykernel_487760/301308756.py:125: DeprecationWarning: Option approximation=False is deprecated as of qiskit-aer 0.13. It will be removed no earlier than 3 months after the release date. Instead, use BackendEstimator from qiskit.primitives.
  main()


AlgorithmError: 'The primitive job to evaluate the energy failed!'

In [3]:
!pip list|grep qiskit

qiskit                2.1.2
qiskit-aer            0.13.3
qiskit-algorithms     0.4.0
qiskit-ibm-runtime    0.41.1
qiskit-nature         0.7.2
